# Analyse 3

We vragen ons af of er meer doelpunten worden gemaakt wanneer 2 ploegen die ver uit elkaar staan in het klassement tegen elkaar spelen, dan in het geval de 2 ploegen dicht bij elkaar staan in het klassement. 
Verdeel voor elk seizoen het klassement denkbeeldig in 3 delen: 
- (1) de eerste 6 ploegen
- (2) de laatste 6 ploegen
- (3) alle ploegen ertussen (eventueel meer of minder dan 6)

Breid de dataframe wedstrijden uit met de positie van de thuisploeg en de uitploeg in het klassement (van de voorgaande speeldag!).
Breid de dataframe wedstrijden uit met het denkbeeldige deel van de thuisploeg en de uitploeg in het klassement: (1), (2) of (3)
Breid de dataframe wedstrijden uit met de categorie waartoe de wedstrijd behoort
- dicht: beide ploegen behoren tot hetzelfde denkbeeldige deel van het klassement (1), (2) of (3)
- gemiddeld: 1 ploeg behoort tot (1) of (2), de andere ploeg behoort tot (3)
- ver: 1 ploeg behoort tot (1) en de andere tot (2)

Bereken over alle seizoenen heen het totaal aantal wedstrijden én het totaal aantal doelpunten voor de 3 categorieën (dicht / gemiddeld / ver). Maak gebruik van wat aan bod komt in Data Science & AI Hoofdstuk 4 (Goodness of fit test) en wat je hiervoor berekend hebt om te berekenen of er significant meer gescoord wordt in de categorie ver dan in de categorie dicht

In [2]:
# Importing the necessary packages
import numpy as np                                  # "Scientific computing"
import scipy.stats as stats                         # Statistical tests

import pandas as pd                                 # Data Frame
from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt                     # Basic visualisation
from statsmodels.graphics.mosaicplot import mosaic  # Mosaic diagram
import seaborn as sns                               # Advanced data visualisation

from scipy.stats import chi2_contingency            # Chi-square test

C:\Users\Matteo\AppData\Local\Temp\ipykernel_26860\1780019501.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd                                 # Data Frame


In [3]:
# Datasets inladen
wedstrijdendata = pd.read_csv("D:/Hogent/Visual Studio Code/DEP/DEP1-2023-2024-groep30/transfermarkt/data/correcte_data/wedstrijden.csv", sep=";")
klassementdata = pd.read_csv("D:/Hogent/Visual Studio Code/DEP/DEP1-2023-2024-groep30/transfermarkt/data/correcte_data/klassement.csv", sep=";")

In [4]:
# De eerste 10 rijen van de dataset bekijken
wedstrijdendata.head(10)

,Seizoen,Speeldag,Datum,Tijdstip,Id,StamnummerThuisploeg,RoepnaamThuisploeg,StamnummerUitploeg,RoepnaamUitploeg,StandThuis,StandUit
0,1960,1,1960-09-04,15:00,4008203,373,Sint-Truidense VV,16,Standard Luik,0,2
1,1960,1,1960-09-04,15:00,4008204,90,Eendracht Aalst,1,Royal Antwerp FC,3,2
2,1960,1,1960-09-04,15:00,4008205,246,Olympic Charleroi,35,RSC Anderlecht,0,7
3,1960,1,1960-09-04,15:00,4008206,13,Beerschot,33,Verviétois,1,0
4,1960,1,1960-09-04,15:00,4008207,3,Club Brugge,10,Union Saint-Gilloise,1,2
5,1960,1,1960-09-04,15:00,4008208,2,Daring Club Brussel,3434,Patro Eisden,2,1
6,1960,1,1960-09-04,15:00,4008209,4,RFC Luik,7,KAA Gent,2,1
7,1960,1,1960-09-04,15:00,4008210,553,Waterschei SV Thor,30,Lierse SK,5,0
8,1960,2,1960-09-11,15:00,4008211,16,Standard Luik,90,Eendracht Aalst,3,0
9,1960,2,1960-09-11,15:00,4008212,35,RSC Anderlecht,4,RFC Luik,2,3


In [5]:
# De eerste 10 rijen van de dataset bekijken
klassementdata.head(10)

,Seizoen,Speeldag,Stand,Stamnummer,Ploeg,AantalGespeeld,AantalGewonnen,AantalGelijk,AantalVerloren,DoelpuntenVoor,DoelpuntenTegen,Doelpuntensaldo,Links_Tweepuntensysteem,Rechts_Tweepuntensysteem,Driepuntensysteem
0,1960,1,1,35,RSC Anderlecht,1,1,0,0,7,0,7,2,0,3
1,1960,1,2,553,Waterschei SV Thor,1,1,0,0,5,0,5,2,0,3
2,1960,1,3,16,Standard Luik,1,1,0,0,2,0,2,2,0,3
3,1960,1,4,90,Eendracht Aalst,1,1,0,0,3,2,1,2,0,3
4,1960,1,5,2,Daring Club Brussel,1,1,0,0,2,1,1,2,0,3
5,1960,1,5,4,RFC Luik,1,1,0,0,2,1,1,2,0,3
6,1960,1,5,10,Union Saint-Gilloise,1,1,0,0,2,1,1,2,0,3
7,1960,1,8,13,Beerschot,1,1,0,0,1,0,1,2,0,3
8,1960,1,9,1,Royal Antwerp FC,1,0,0,1,2,3,-1,0,2,0
9,1960,1,10,3,Club Brugge,1,0,0,1,1,2,-1,0,2,0


In [12]:
# Functie om segment te bepalen
def segment_bepalen(positie, totaal_teams):
    if positie <= 6:
        return 1  # Top 6 teams
    elif positie > totaal_teams - 6:
        return 2  # Laatste 6 teams
    else:
        return 3  # Midden teams

# Functie om categorie te bepalen
def bepaal_categorie(deelklassement_thuis, deelklassement_uit):
    if deelklassement_thuis == deelklassement_uit:
        return 'dicht'
    elif (deelklassement_thuis in {1, 2} and deelklassement_uit == 3) or (deelklassement_uit in {1, 2} and deelklassement_thuis == 3):
        return 'gemiddeld'
    elif (deelklassement_thuis == 1 and deelklassement_uit == 2) or (deelklassement_thuis == 2 and deelklassement_uit == 1):
        return 'ver'
    return 'niet gedefinieerd'  # Voor onverwachte combinaties

def verwerk_en_categoriseer_wedstrijden(wedstrijden, klassement):
    # Omzetten van 'Seizoen' en 'Speeldag' naar strings voor consistentie
    wedstrijden['Seizoen'] = wedstrijden['Seizoen'].astype(str)
    klassement['Seizoen'] = klassement['Seizoen'].astype(str)
    wedstrijden['Speeldag'] = wedstrijden['Speeldag'].astype(int)
    klassement['Speeldag'] = klassement['Speeldag'].astype(int)

    # Kopieer klassement en pas de speeldag aan naar de vorige speeldag
    klassement_vorige_speeldag = klassement.copy()
    klassement_vorige_speeldag['Speeldag'] += 1

    # Samenvoegen op basis van seizoen, speeldag, en stamnummer voor thuisploeg
    wedstrijden = pd.merge(wedstrijden, klassement_vorige_speeldag[['Seizoen', 'Speeldag', 'Stamnummer', 'Stand']], left_on=['Seizoen', 'Speeldag', 'StamnummerThuisploeg'], right_on=['Seizoen', 'Speeldag', 'Stamnummer'], how='left')
    wedstrijden.rename(columns={'Stand': 'PositieKlassementThuisploeg'}, inplace=True)

    # Samenvoegen op basis van seizoen, speeldag, en stamnummer voor uitploeg
    wedstrijden = pd.merge(wedstrijden, klassement_vorige_speeldag[['Seizoen', 'Speeldag', 'Stamnummer', 'Stand']], left_on=['Seizoen', 'Speeldag', 'StamnummerUitploeg'], right_on=['Seizoen', 'Speeldag', 'Stamnummer'], how='left')
    wedstrijden.rename(columns={'Stand': 'PositieKlassementUitploeg'}, inplace=True)

    # Zorg dat posities integers zijn
    wedstrijden['PositieKlassementThuisploeg'] = wedstrijden['PositieKlassementThuisploeg'].fillna(0).astype(int)
    wedstrijden['PositieKlassementUitploeg'] = wedstrijden['PositieKlassementUitploeg'].fillna(0).astype(int)

    # berekenen van deel klassement
    wedstrijden['DeelKlassementThuisploeg'] = wedstrijden.apply(lambda x: segment_bepalen(x['PositieKlassementThuisploeg'], x['StandThuis']), axis=1)
    wedstrijden['DeelKlassementUitploeg'] = wedstrijden.apply(lambda x: segment_bepalen(x['PositieKlassementUitploeg'], x['StandUit']), axis=1)

    # Toevoegen categorieën voor deel klassement
    wedstrijden['Categorie'] = wedstrijden.apply(lambda row: bepaal_categorie(row['DeelKlassementThuisploeg'], row['DeelKlassementUitploeg']), axis=1)

    # Filter om alleen de benodigde kolommen te behouden
    output_fields = [
        'Seizoen', 'Speeldag', 'Datum', 'Tijdstip', 'Id', 'StamnummerThuisploeg', 'RoepnaamThuisploeg', 
        'StamnummerUitploeg', 'RoepnaamUitploeg', 'StandThuis', 'StandUit', 'PositieKlassementThuisploeg', 
        'PositieKlassementUitploeg','DeelKlassementThuisploeg', 'DeelKlassementUitploeg', 'Categorie'
    ]
    wedstrijden = wedstrijden[output_fields]

    return wedstrijden

# uitvoeren van de functie
wedstrijden = verwerk_en_categoriseer_wedstrijden(wedstrijdendata, klassementdata)
wedstrijden

,Seizoen,Speeldag,Datum,Tijdstip,Id,StamnummerThuisploeg,RoepnaamThuisploeg,StamnummerUitploeg,RoepnaamUitploeg,StandThuis,StandUit,PositieKlassementThuisploeg,PositieKlassementUitploeg,DeelKlassementThuisploeg,DeelKlassementUitploeg,Categorie
0,1960,1,1960-09-04,15:00,4008203,373,Sint-Truidense VV,16,Standard Luik,0,2,0,0,1,1,dicht
1,1960,1,1960-09-04,15:00,4008204,90,Eendracht Aalst,1,Royal Antwerp FC,3,2,0,0,1,1,dicht
2,1960,1,1960-09-04,15:00,4008205,246,Olympic Charleroi,35,RSC Anderlecht,0,7,0,0,1,1,dicht
3,1960,1,1960-09-04,15:00,4008206,13,Beerschot,33,Verviétois,1,0,0,0,1,1,dicht
4,1960,1,1960-09-04,15:00,4008207,3,Club Brugge,10,Union Saint-Gilloise,1,2,0,0,1,1,dicht
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17901,2023,30,2024-03-17,16:00,4098846,1,Royal Antwerp FC,10,Union Saint-Gilloise,1,1,4,1,1,1,dicht
17902,2023,30,2024-03-17,18:30,4098845,7,KAA Gent,22,R Charleroi SC,5,0,8,12,2,2,dicht
17903,2023,30,2024-03-17,18:30,4098847,18,Oud-Heverlee Leuven,25,KV Mechelen,1,0,13,6,2,1,ver
17904,2023,30,2024-03-17,18:30,4098848,12,Cercle Brugge,5479,RWDM,4,0,7,15,2,2,dicht


In [14]:
def goodness_of_fit_test(wedstrijden):
    categories = ['dicht', 'gemiddeld', 'ver']
    observed = []
    
    # Zorg dat 'Categorie' gebruikt wordt als index als dit nog niet het geval is
    if 'Categorie' in wedstrijden.columns:
        wedstrijden.set_index('Categorie', inplace=True)

    for category in categories:
        if category in wedstrijden.index:
            # Verzamel de totale doelpunten en totale wedstrijden voor elke categorie
            goals = wedstrijden.loc[category, 'TotaalDoelpunten']
            non_goals = wedstrijden.loc[category, 'TotaalWedstrijden'] * 2 - goals  # Vermenigvuldig met 2 voor thuis en uit
            # Zorg dat er voldoende data is voor de test
            if non_goals >= 0 and (goals + non_goals > 5):
                observed.append([goals, non_goals])
            else:
                return {'error': f'Niet genoeg gegevens voor categorie {category}'}
        else:
            return {'error': f'Categorie {category} ontbreekt in de data'}

    if len(observed) > 1:
        # Voer de chi-kwadraat toets uit
        chi2, p, dof, expected = chi2_contingency(observed)
        return {'chi2_statistic': chi2, 'p_value': p, 'degrees_of_freedom': dof, 'expected_frequencies': expected.tolist()}
    else:
        return {'error': 'Niet genoeg categorieën met voldoende gegevens voor een geldige chi-kwadraattoets'}
    
# Uitvoeren van de functie
test_resultaten = goodness_of_fit_test(wedstrijden)

# Print de resultaten
print(test_resultaten)

KeyError: 'TotaalDoelpunten'